<a href="https://colab.research.google.com/github/prinzessinmarlenifee/SenseCap/blob/main/SensecCap_v1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Hier sollte nachdem mit einer session getestet wurde, das für alle sessions, mit 1 fused excel sheet für die drei sensoren getestet werden.
sprich, hot labels nutzten, und ein excel sheet mit allen drei sensors schon gefused

In [ ]:
import os
import json
import numpy as np
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

# --- PARAMETER ---
sampling_rate = 60  # wie besprochen, evtl step size anpassen
window_size = 60
step_size = 11

# Ordner, in dem deine 18 Sessions liegen
base_dir = '/content/drive/MyDrive/IMU_Sessions/'  # ANPASSEN

# Alle Session-Ordner suchen (z. B. 001, 002, ..., 018)
session_dirs = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])

def parse_hot_labels(json_path, total_frames):
    with open(json_path, 'r') as f:
        data = json.load(f)
    button_press_str = data['button_presses']
    entries = button_press_str.strip().split(';')

    label_changes = []
    for entry in entries:
        if ':' in entry:
            label, frame = entry.strip().split(':')
            label = label.strip()
            if label == 'Peadling':
                label = 'Pedaling'  # ✅ Korrektur hier
            label_changes.append((int(frame.strip()), label))

    frame_labels = ['Unknown'] * total_frames
    for i, (start_frame, label) in enumerate(label_changes):
        end_frame = label_changes[i + 1][0] if i + 1 < len(label_changes) else total_frames
        for f in range(start_frame, min(end_frame, total_frames)):
            frame_labels[f] = label
    return frame_labels

def window_data(imu_data, frame_labels):
    X_windows, y_windows = [], []
    for start in range(0, len(frame_labels) - window_size + 1, step_size):
        end = start + window_size
        window = imu_data[start:end]
        label_window = frame_labels[start:end]

        label_counts = Counter(label_window)
        dominant_label = label_counts.most_common(1)[0][0]
        if dominant_label == 'Unknown':
            continue
        X_windows.append(window)
        y_windows.append(dominant_label)
    return np.array(X_windows), np.array(y_windows)

# --- DATEN SAMMELN ---
all_X, all_y = [], []
sessions_X, sessions_y = [], []

for sess_dir in session_dirs:
    print(f"Lade Session {sess_dir}...")
    session_path = os.path.join(base_dir, sess_dir)
    hot_path = os.path.join(session_path, 'hot.json')
    imu_path = os.path.join(session_path, 'imu.csv')  # ← ANPASSEN je nach Format

    # Lade IMU-Daten (angenommen CSV, mit 6 Spalten)
    imu_data = np.loadtxt(imu_path, delimiter=',')  # oder pd.read_csv()

    total_frames = imu_data.shape[0]
    frame_labels = parse_hot_labels(hot_path, total_frames)
    X, y = window_data(imu_data, frame_labels)

    sessions_X.append(X)
    sessions_y.append(y)

# --- LEAVE-ONE-SESSION-OUT ---
for test_idx in range(len(sessions_X)):
    print(f"\n📌 Teste auf Session {test_idx+1}/{len(sessions_X)}")

    X_test = sessions_X[test_idx]
    y_test = sessions_y[test_idx]

    X_train = np.concatenate([x for i, x in enumerate(sessions_X) if i != test_idx])
    y_train = np.concatenate([y for i, y in enumerate(sessions_y) if i != test_idx])

    # Label Encoding
    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train)
    y_test_enc = le.transform(y_test)

    # Modell
    model = models.Sequential([
        layers.Conv1D(64, 3, activation='relu', input_shape=X_train.shape[1:]),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(0.3),
        layers.LSTM(64),
        layers.Dropout(0.3),
        layers.Dense(100, activation='relu'),
        layers.Dense(len(le.classes_), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Training
    history = model.fit(X_train, y_train_enc,
                        validation_split=0.1,
                        epochs=20,
                        batch_size=32,
                        verbose=0)

    # Test
    test_loss, test_acc = model.evaluate(X_test, y_test_enc, verbose=0)
    print(f"✅ Test-Accuracy auf Session {test_idx+1}: {test_acc:.2f}")


🧰 Erweiterter Code-Block mit Bericht

Füge diesen Teil nach der Modellbewertung in deiner for test_idx in range(...)-Schleife ein:

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

all_accuracies = []
all_reports = []

for test_idx in range(len(sessions_X)):
    print(f"\n📌 Teste auf Session {test_idx+1}/{len(sessions_X)}")

    # [ ... gleicher Trainingscode wie zuvor ... ]

    test_loss, test_acc = model.evaluate(X_test, y_test_enc, verbose=0)
    print(f"✅ Test-Accuracy auf Session {test_idx+1}: {test_acc:.2f}")
    all_accuracies.append(test_acc)

    # Klassifikationsbericht
    y_pred = model.predict(X_test, verbose=0)
    y_pred_classes = np.argmax(y_pred, axis=1)

    report = classification_report(y_test_enc, y_pred_classes, target_names=le.classes_, output_dict=True)
    all_reports.append(report)

    # Confusion Matrix anzeigen (optional)
    cm = confusion_matrix(y_test_enc, y_pred_classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
    disp.plot(xticks_rotation=45)
    plt.title(f"Confusion Matrix – Session {test_idx+1}")
    plt.show()


🧾 Zusammenfassung nach dem Training:

Nach der Schleife kannst du am Ende folgendes hinzufügen, um einen Bericht zu erzeugen:

In [ ]:
# Durchschnittliche Accuracy über alle Sessions
mean_acc = np.mean(all_accuracies)
print("\n📈 Zusammenfassung:")
for i, acc in enumerate(all_accuracies):
    print(f"  Session {i+1}: Accuracy = {acc:.2f}")
print(f"\n✅ Durchschnittliche Test-Accuracy über alle Sessions: {mean_acc:.2f}")


📊 Bonus: CSV speichern (optional)

In [ ]:
import pandas as pd

summary = {'Session': [], 'Accuracy': []}
for i, acc in enumerate(all_accuracies):
    summary['Session'].append(f"Session_{i+1}")
    summary['Accuracy'].append(acc)

df_summary = pd.DataFrame(summary)
df_summary.to_csv("session_accuracy_report.csv", index=False)
print("📁 Bericht gespeichert als session_accuracy_report.csv")
